In [ ]:
from openai import AzureOpenAI
import os
# print(os.getcwd())
os.chdir("..")
# print(os.getcwd())

from config import Config

envConfig = Config()

client = AzureOpenAI(
    api_key=envConfig.openai_api_key,
    api_version=envConfig.openai_api_version,
    azure_endpoint=envConfig.openai_api_base_url
)

In [ ]:
from langchain_openai.chat_models import AzureChatOpenAI

model = AzureChatOpenAI(
    api_key=envConfig.openai_api_key,
    api_version=envConfig.openai_api_version,
    azure_endpoint=envConfig.openai_api_base_url,
    azure_deployment="gpt-turbo"
)

prompt = "I want to open a resturant for Indian food, suggest a short and cool name for it"
model.invoke(prompt)

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke(prompt)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt_resturant_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a resturant for {cuisine} food, suggest a short and cool name for it"
)

prompt_resturant_name.format(cuisine="Indian")

chain_resturant_name = LLMChain(llm =  model, prompt = prompt_resturant_name)

resturant_name = chain_resturant_name.run({
    "cuisine":"Indian"
})

print(resturant_name)

In [ ]:
prompt_menu_items = PromptTemplate(
    input_variables=['resturant'],
    template="Please suggest some food menu for the resturant {resturant}, as a comma separated list."
)

chain_food_menu = LLMChain(llm =  model, prompt = prompt_menu_items)

chain_food_menu.run({
    "resturant":resturant_name
})

Simple Sequential Chain

In [ ]:

from langchain.chains import SimpleSequentialChain

# Create a SimpleSequentialChain with the defined prompts and chain
simple_seq_chain = SimpleSequentialChain(chains=[chain_resturant_name, chain_food_menu])

simple_seq_chain.run("Thai")


Sequential Chain ( Stateful)

In [ ]:
from langchain.chains import SequentialChain

prompt_resturant_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a resturant for {cuisine} food, suggest a short and cool name for it"
)

chain_rest_name = LLMChain(llm =  model, prompt = prompt_resturant_name, output_key="resturant_name")


prompt_menu_items = PromptTemplate(
    input_variables=['resturant_name'],
    template="Please suggest some food menu for the resturant {resturant_name}, as a comma separated list."
)

chain_food_menu = LLMChain(llm =  model, prompt = prompt_menu_items, output_key="menu_items")

seq_chain = SequentialChain(
    chains=[chain_rest_name, chain_food_menu],
    input_variables=['cuisine'],
    output_variables=["resturant_name", "menu_items"]

)

seq_chain.invoke("Indian")